<a href="https://colab.research.google.com/github/jacksonkarel/etl-langchain/blob/main/langchain_karl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import json
keys = json.load(open('keys.json'))
os.environ["OPENAI_API_KEY"] = keys['openai']
os.environ["SERPAPI_API_KEY"] = keys['serp']
import pandas as pd
import numpy as np

In [4]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.utilities import PythonREPL
from langchain.agents import initialize_agent

In [5]:
python_repl = PythonREPL()

tools = [
    Tool(
        name="Python REPL",
        func=python_repl.run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. Always print the output of the python command."
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

llm=OpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="conversational-react-description", verbose=True, memory=memory)

In [6]:
input = """Execute the following steps to run a machine learning ETL on `dataset.csv`

1. Using python and pandas to load `dataset.csv`

2. Using python, print out each column name and its datatype

3. Using python, 
"""

agent_chain.run(input=input)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Python REPL
Action Input: 
import pandas as pd
df = pd.read_csv('dataset.csv')
for col in df.columns:
    print(col, df[col].dtype)
Observation: 
Thought: Do I need to use a tool? No
AI: The output of the Python REPL should be a list of column names and their corresponding data types.

> Finished chain.


'The output of the Python REPL should be a list of column names and their corresponding data types.'